In [36]:
import pandas as pd
import numpy as np
from datetime import datetime
regularseason = pd.read_csv('MDataFiles_Stage1/MRegularSeasonDetailedResults.csv')
postseason = pd.read_csv('MDataFiles_Stage1/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('MDataFiles_Stage1/MMasseyOrdinals.csv')
kenpom = rankings[(rankings['SystemName'] == 'POM') & (rankings['RankingDayNum'] >= 128)]
import time

In [37]:
postseason

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,2019,146,1120,77,1246,71,N,1,26,65,...,21,12,21,11,30,14,14,5,5,19
1111,2019,146,1277,68,1181,67,N,0,30,70,...,21,8,13,13,29,14,17,4,9,9
1112,2019,152,1403,61,1277,51,N,0,22,51,...,24,14,18,8,28,6,11,1,2,15
1113,2019,152,1438,63,1120,62,N,0,25,51,...,31,11,14,9,24,9,5,3,3,12


In [38]:
regularseason

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92827,2020,128,1389,56,1233,54,N,0,15,60,...,16,18,20,4,27,6,17,8,4,20
92828,2020,128,1393,81,1314,53,N,0,28,61,...,16,11,13,13,29,9,18,0,1,18
92829,2020,128,1402,85,1111,68,H,0,34,57,...,15,25,35,8,17,6,18,7,2,18
92830,2020,128,1412,74,1427,69,N,0,26,53,...,27,7,7,12,22,11,13,4,2,18


In [39]:
def add_rate_cols(df):
    
    df['possessions'] = .5 * (df['FGA'] + (.475 * df['FTA']) - df['OR'] + df['TO'])
    df['eFG'] = (df['FGM'] + (.5 * df['FGM3'])) / df['FGA']
    df['TS%'] = df['Score'] / (2 * (df['FGA'] + .475 * df['FTA']))
    df['FTAr'] = df['FTA'] / df['FGA']
    df['3PAr'] = df['FGA3'] / (df['FGA'] + (.475*df['FTA']))
    df['OR%'] = df['OR'] / (df['OR'] + df['Opp_DR'])
    df['DR%'] = df['DR'] / (df['DR'] + df['Opp_OR'])
    df['REB%'] = (df['OR'] + df['DR']) / (df['OR'] + df['Opp_DR'] + df['Opp_OR'] + df['DR'])
    df['TR'] = (df['OR'] + df['DR'])
    df['ATOr'] = df['Ast'] / df['TO']
    df['Ast%'] = df['Ast'] / df['FGM']
    df['Stl%'] = df['Stl'] / (df['Opp_FGA'] + .475*df['Opp_FTA'] + df['Opp_TO'])
    df['Blk%'] = df['Blk'] / (df['Opp_FGA'])
    df['TO_ratio'] = df['TO'] / (df['FGA'] + .475*df['FTA'] + df['TO'])
    df['Opp_eFG'] = (df['Opp_FGM'] + (.5 * df['Opp_FGM3'])) / df['Opp_FGA']
    df['Opp_TSpct'] = df['Opp_Score'] / (2 * (df['Opp_FGA'] + .475 * df['Opp_FTA']))
    df['Opp_FTR'] = df['Opp_FTA'] / df['Opp_FGA']
    df['Opp_3PR'] = df['Opp_FGA3'] / (df['Opp_FGA'] + (.475*df['Opp_FTA']))
    df['Opp_TR'] = (df['Opp_OR'] + df['Opp_DR'])
    df['Opp_ATOr'] = df['Opp_Ast'] / df['Opp_TO']
    df['Opp_Ast%'] = df['Opp_Ast'] / df['Opp_FGM']
    df['Opp_Stl%'] = df['Opp_Stl'] / (df['FGA'] + .475*df['FTA'] + df['TO'])
    df['Opp_Blk%'] = df['Opp_Blk'] / (df['FGA'])
    df['Opp_TO_ratio'] = df['Opp_TO'] / (df['Opp_FGA'] + .475*df['Opp_FTA'] + df['Opp_TO'])

    return df

In [40]:
def team_regular_season(team_id,Season = all, DayNum = all, average = True):
    
    teamwins = regularseason[(regularseason['WTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)] 
    teamlosses = regularseason[(regularseason['LTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)]
    
    teamwinsoffense = teamwins.filter(like = 'W')    
    teamlossesoffense = teamlosses.filter(like = 'L')

    teamwinsoffense.columns = teamwinsoffense.columns.str.lstrip('W')
    teamlossesoffense.columns = teamlossesoffense.columns.str.lstrip('L')

    teamoffense = pd.concat([teamwinsoffense, teamlossesoffense])

    teamwinsopponents = teamwins.filter(like = 'L')
    teamlossesopponents = teamlosses.filter(like = 'W')

    teamwinsopponents.columns = teamwinsopponents.columns.str.lstrip('L')
    teamlossesopponents.columns = teamlossesopponents.columns.str.lstrip('W')

    teamopponents = pd.concat([teamwinsopponents, teamlossesopponents])
    
    teamtotal = pd.concat([teamoffense, teamopponents.add_prefix('Opp_')], axis = 1)
    
    ###cols_to_drop = ['Opp_TeamID', ]
    
    if average == True:
        teamtotal = teamtotal.groupby('TeamID').mean().reset_index()
        teamtotal = add_rate_cols(teamtotal)
    
    if average == False:
        teamtotal = add_rate_cols(teamtotal)
        
    return teamtotal

In [41]:
start = time.time()
marquette = team_regular_season(1266, 2020, 132, True)
end = time.time()
print('Execution time = %.6f seconds' % (end-start))

Execution time = 0.046910 seconds


In [42]:
kenpom

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
127259,2003,133,POM,1102,160
127260,2003,133,POM,1103,163
127261,2003,133,POM,1104,33
127262,2003,133,POM,1105,307
127263,2003,133,POM,1106,263
...,...,...,...,...,...
4113090,2020,128,POM,1463,57
4113091,2020,128,POM,1464,232
4113092,2020,128,POM,1465,202
4113093,2020,128,POM,1466,281


In [43]:
def get_kenpom(team_id, year):
    rank = kenpom[(kenpom['TeamID'] == team_id) & (kenpom['Season'] == year)]
    rank.reset_index(inplace = True)
    print(rank)
    return rank.OrdinalRank

In [44]:
get_kenpom(1181, 2020)

     index  Season  RankingDayNum SystemName  TeamID  OrdinalRank
0  4112816    2020            128        POM    1181            5


0    5
Name: OrdinalRank, dtype: int64

In [45]:
def single_game(team1_id, team2_id, year = 2020, daynum = 132):
    team1 = team_regular_season(team1_id, year, daynum)
    team2 = team_regular_season(team2_id, year, daynum)
    team1Pom = get_kenpom(team1_id, year)
    team2Pom = get_kenpom(team2_id, year)
    
    fav_win = team1Pom.lt(team2Pom)
    
    game = pd.concat([team1.add_prefix('W_'), team2.add_prefix('L_'), fav_win], axis = 1)
    game.rename(columns={game.columns[108]: 'FavWin'}, inplace = True)
    
    return game



In [46]:
st = time.time()
duke_unc = single_game(1181, 1314, 2020)
duke_unc
ed = time.time()
print('Execution time = %.6f seconds' % (ed - st))

Execution time = 0.112215 seconds


In [47]:
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTR,L_Opp_3PR,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_pct,L_Opp_Stl_pct,L_Opp_Blk_pct,L_Opp_TO_ratio,FavWin
0,1181,82.516129,29.677419,63.129032,7.0,19.903226,16.16129,22.451613,12.709677,27.419355,...,0.527685,0.2915,0.361892,34.30303,1.203704,0.539739,0.081458,0.067055,0.142377,True


In [53]:
schedule = regularseason[(regularseason['Season'] == 2017) & (regularseason['DayNum'] >= 12)]
schedule.reset_index(inplace = True)
matchups = list(zip(schedule.WTeamID, schedule.LTeamID, schedule.Season, schedule.DayNum))
len(matchups)

5281

In [54]:
regularseason[(regularseason['Season'] == 2017) & (regularseason['DayNum'] >= 12)]

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
71355,2017,12,1139,89,1294,52,H,0,32,60,...,20,16,28,7,17,3,19,3,0,22
71356,2017,12,1172,86,1111,74,H,0,30,63,...,22,21,30,13,26,10,14,6,5,25
71357,2017,12,1181,96,1213,61,H,0,32,60,...,12,6,12,10,21,11,21,7,2,26
71358,2017,12,1199,88,1149,67,H,0,33,67,...,25,12,19,8,22,11,15,3,6,19
71359,2017,12,1207,105,1367,60,H,0,36,54,...,25,12,28,20,13,11,26,6,2,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76631,2017,132,1276,71,1458,56,N,0,27,48,...,15,6,8,14,18,10,15,4,3,13
76632,2017,132,1343,71,1463,59,N,0,25,52,...,20,13,19,14,20,12,7,4,5,13
76633,2017,132,1348,70,1433,63,N,0,24,54,...,14,17,22,23,24,8,5,4,1,16
76634,2017,132,1374,71,1153,56,N,0,26,52,...,24,14,18,17,22,7,7,7,1,13


In [55]:
start_time = time.time()
appended_data = []
for i in range(len(schedule)):
    data = single_game(matchups[i][0], matchups[i][1], daynum = matchups[i][2])
    appended_data.append(data)

games17 = pd.concat(appended_data, axis=0)

end_time = time.time()
print('Execution time = %.6f seconds' % (end_time-start_time))

KeyboardInterrupt: 

In [33]:
games2020

,1TeamID,1Score,1FGM,1FGA,1FGM3,1FGA3,1FTM,1FTA,1OR,1DR,...,2Opp_ORpct,2Opp_DRpct,2Opp_REBpct,2Opp_TR,2Opp_ATOr,2Opp_Ast_pct,2Opp_Stl_pct,2Opp_Blk_pct,2Opp_TO_ratio,FavWin
0,1111.0,65.000000,22.333333,54.333333,5.000000,17.333333,15.333333,22.000000,8.333333,28.000000,...,0.353982,0.747748,0.549107,41.000000,1.047619,0.523810,0.110328,0.075581,0.163679,True
0,1125.0,79.950000,30.300000,64.400000,10.150000,28.300000,9.200000,13.200000,9.350000,30.000000,...,0.279743,0.725434,0.514460,35.578947,1.160156,0.563567,0.096970,0.057325,0.171114,True
0,1125.0,80.321429,30.035714,63.071429,9.857143,27.714286,10.392857,14.678571,8.821429,28.107143,...,0.282828,0.718026,0.512208,35.740741,1.178378,0.595628,0.089398,0.056365,0.173817,True
0,1286.0,66.333333,22.666667,50.000000,6.666667,16.333333,14.333333,21.666667,6.000000,30.000000,...,0.352941,0.744681,0.552347,38.250000,0.947368,0.500000,0.117956,0.082192,0.175587,True
0,1287.0,66.833333,25.083333,59.000000,5.750000,20.916667,10.916667,18.083333,10.250000,26.416667,...,0.289062,0.731707,0.528144,36.750000,1.069767,0.562691,0.098657,0.054598,0.183125,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1285.0,69.850000,26.550000,58.500000,5.950000,16.650000,10.800000,15.200000,7.300000,24.850000,...,0.283708,0.734332,0.512448,39.000000,1.075188,0.557505,0.089846,0.056067,0.163268,False
0,1225.0,62.904762,23.380952,54.238095,4.571429,14.142857,11.571429,17.476190,7.571429,25.904762,...,0.277778,0.737828,0.509102,38.619048,1.071672,0.563734,0.088853,0.055046,0.163047,False
0,1226.0,64.500000,22.000000,51.875000,7.375000,22.625000,13.125000,20.750000,10.250000,21.000000,...,0.296588,0.717884,0.511568,39.800000,1.020408,0.543478,0.090858,0.056061,0.165461,False
0,1276.0,82.500000,31.333333,59.500000,10.000000,24.500000,9.833333,14.166667,7.333333,29.000000,...,0.221865,0.651852,0.421687,30.625000,0.686441,0.440217,0.074638,0.039337,0.182967,False


In [57]:
postseason17 = postseason[postseason.Season == 2017]
postseason17

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
914,2017,134,1243,95,1448,88,N,0,31,47,...,27,29,32,7,14,14,11,8,1,25
915,2017,134,1291,67,1309,66,N,0,25,47,...,9,7,12,10,18,12,14,10,2,18
916,2017,135,1413,67,1300,63,N,0,22,48,...,26,10,13,9,21,14,9,10,0,15
917,2017,135,1425,75,1344,71,N,0,24,57,...,20,12,17,7,20,17,9,3,2,21
918,2017,136,1112,100,1315,82,N,0,36,62,...,22,8,16,9,22,18,6,6,2,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976,2017,146,1314,75,1246,73,N,0,27,58,...,19,12,19,11,23,16,9,7,2,19
977,2017,146,1376,77,1196,70,N,0,26,53,...,26,13,14,13,18,11,16,7,5,21
978,2017,152,1211,77,1376,73,N,0,29,60,...,20,16,22,13,23,12,5,7,6,14
979,2017,152,1314,77,1332,76,N,0,25,68,...,26,25,28,12,31,7,16,5,7,20


In [58]:
postseason_sch = list(zip(postseason17.WTeamID, postseason17.LTeamID, postseason17.DayNum, postseason17.Season))
postseason_sch

[(1243, 1448, 134, 2017),
 (1291, 1309, 134, 2017),
 (1413, 1300, 135, 2017),
 (1425, 1344, 135, 2017),
 (1112, 1315, 136, 2017),
 (1139, 1457, 136, 2017),
 (1196, 1190, 136, 2017),
 (1199, 1195, 136, 2017),
 (1211, 1355, 136, 2017),
 (1235, 1305, 136, 2017),
 (1292, 1278, 136, 2017),
 (1321, 1435, 136, 2017),
 (1323, 1343, 136, 2017),
 (1345, 1436, 136, 2017),
 (1388, 1433, 136, 2017),
 (1437, 1291, 136, 2017),
 (1438, 1423, 136, 2017),
 (1452, 1137, 136, 2017),
 (1458, 1439, 136, 2017),
 (1462, 1268, 136, 2017),
 (1116, 1371, 137, 2017),
 (1124, 1308, 137, 2017),
 (1153, 1243, 137, 2017),
 (1181, 1407, 137, 2017),
 (1242, 1413, 137, 2017),
 (1246, 1297, 137, 2017),
 (1257, 1240, 137, 2017),
 (1276, 1329, 137, 2017),
 (1277, 1274, 137, 2017),
 (1314, 1411, 137, 2017),
 (1332, 1233, 137, 2017),
 (1348, 1166, 137, 2017),
 (1376, 1266, 137, 2017),
 (1417, 1245, 137, 2017),
 (1425, 1374, 137, 2017),
 (1455, 1173, 137, 2017),
 (1112, 1388, 138, 2017),
 (1139, 1292, 138, 2017),
 (1196, 1438

In [67]:
start_time = time.time()
appended_data = []
for i in range(len(postseason_sch)):
    data = single_game(postseason_sch[i][0], postseason_sch[i][1], daynum = postseason_sch[i][2])
    appended_data.append(data)

bracket17 = pd.concat(appended_data, axis=0)

end_time = time.time()
print('Execution time = %.6f seconds' % (end_time-start_time))

Execution time = 4.694279 seconds


In [68]:
bracket17

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTR,L_Opp_3PR,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_pct,L_Opp_Stl_pct,L_Opp_Blk_pct,L_Opp_TO_ratio,FavWin
0,1243.0,64.187500,22.531250,54.437500,6.656250,21.281250,12.468750,18.875000,10.343750,22.468750,...,0.531961,0.352060,0.302537,35.032258,1.207101,0.498168,0.091235,0.062747,0.134151,True
0,1291.0,63.806452,22.322581,55.451613,6.645161,21.258065,12.516129,18.096774,10.677419,24.032258,...,0.594291,0.397288,0.282887,35.038462,1.075130,0.563094,0.083872,0.079092,0.180429,True
0,1413.0,70.666667,24.533333,54.533333,7.966667,20.333333,13.633333,17.366667,7.200000,22.666667,...,0.519685,0.337121,0.333578,35.285714,0.783784,0.516320,0.078405,0.055629,0.194596,True
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
0,1425.0,70.290323,25.193548,57.419355,6.451613,18.419355,13.451613,20.322581,10.741935,27.774194,...,0.515016,0.373743,0.313379,35.516129,0.780172,0.513475,0.083546,0.056695,0.188987,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1314.0,72.212121,26.212121,62.363636,5.484848,18.030303,14.242424,20.878788,13.757576,28.121212,...,0.488806,0.320506,0.332550,32.451613,0.900538,0.467225,0.063423,0.056878,0.150734,False
0,1376.0,73.387097,26.838710,60.967742,5.612903,18.096774,14.096774,22.451613,11.935484,25.419355,...,0.519233,0.337257,0.306924,31.709677,0.865000,0.483240,0.071510,0.043956,0.168693,False
0,1211.0,87.424242,31.757576,61.666667,7.454545,19.303030,16.454545,23.909091,11.303030,28.727273,...,0.515826,0.530857,0.318205,35.419355,0.738589,0.542683,0.072662,0.059259,0.187414,True
0,1314.0,72.212121,26.212121,62.363636,5.484848,18.030303,14.242424,20.878788,13.757576,28.121212,...,0.509763,0.294944,0.356267,33.290323,0.918854,0.531768,0.051978,0.060274,0.171134,False


In [49]:
bracket18.to_csv('./data/bracket18.csv')